In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from kerastuner import RandomSearch
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import json

# Function to load configurations
def load_config(config_file):
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config

# Load the configuration
config = load_config('config/config.json')

# Access the dataset path
dataset_dir = config['dataset_path']
train_dir = config['train_path']
val_dir = config['val_path']
test_dir = config['test_path']

In [ ]:
def build_model(hp):
    # Define the model
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(256, 256, 3)),  # First convolutional layer
        MaxPooling2D(2,2),  # First pooling layer reduces spatial dimensions by half
        Conv2D(64, (3,3), activation='relu'),  # Second convolutional layer
        MaxPooling2D(2,2),  # Second pooling layer
        Conv2D(128, (3,3), activation='relu'),  # Third convolutional layer
        MaxPooling2D(2,2),  # Third pooling layer
        Flatten(),  # Flatten the output of the last pooling layer
        Dense(131072, activation='relu'),  # Adjust this number to match the flattened output
        Dropout(0.5),  # Dropout layer to reduce overfitting
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])


    
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    #model.summary()
    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Adjust the number of trials here
    executions_per_trial=1,
    directory='model_tuning',
    project_name='MushroomClassification'
)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
number_of_epochs = 40

In [ ]:
tuner.search(
    train_generator,
    epochs=number_of_epochs,
    validation_data=val_generator
)

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)

In [ ]:
# Train the best model
model.fit(
    train_generator,
    epochs=number_of_epochs,
    validation_data=val_generator
)

In [ ]:
loss, accuracy = model.evaluate(val_generator)
print(f"Final validation accuracy: {accuracy*100:.2f}%")

In [ ]:
# Save the model, optional
model.save(f'Pth_Files/CNN_model_test_{number_of_epochs}.keras')